In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import pandas as pd
# read the dataset in from the excel file
df_t = pd.read_csv('~/CIS_4496_Project/data/raw/transactions_train.csv')
# orders = pd.read_csv('~/CIS_4496_Project/data/df_ct/df_ct.csv')
# show the first few rows
df_t.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [107]:
import polars as pl
from scripts import df_ct_combine
orders = df_ct_combine.combine()

In [4]:
orders

,customer_id,articles
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"[663713001, 541518023, 663713001, 578020002, 7..."
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,"[505221004, 685687003, 685687004, 685687001, 5..."
2,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,"[688873012, 501323011, 598859003, 688873020, 6..."
3,0008968c0d451dbc5a9968da03196fe20051965edde741...,"[531310002, 529841001, 662980002, 496111003, 5..."
4,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,"[501820043, 501820043, 674681001, 671505001, 6..."
...,...,...
1362276,fece2f68864c311a0b5208e2eb735b3dcde7e41461d327...,"[756322001, 822311012, 891763003, 889133001]"
1362277,fee56cc5315dafb35a4490ccc6f711092cae913550c832...,"[903647001, 903647001]"
1362278,ff5b8a8b26bf93a66290e9bd1b73393ac6a58968a78519...,"[913597001, 903420001, 761406001]"
1362279,b4db5e5259234574edfff958e170fe3a5e13b6f146752c...,"[673186001, 717205001, 669713004, 514134001, 5..."


In [5]:
type(orders['articles'][0])

numpy.ndarray

In [6]:
orders['articles'] = orders['articles'].apply(list)
type(orders['articles'][0])

list

In [7]:
type(orders['articles'][0][0])

str

In [8]:
for i in range(len(orders)):
    orders.loc[i] = [orders['customer_id'][i], list(map(int, orders['articles'][i]))]

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3199: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


In [9]:
type(orders['articles'][0][0])

int

In [10]:
# type(orders['articles'][0])

In [11]:
# from ast import literal_eval
# orders['articles'] = orders['articles'].apply(literal_eval)
# type(orders['articles'][0])

In [12]:
for c, alist in zip(orders['customer_id'], orders['articles']):    
    if ('...' in alist):
        print(c)

In [13]:
!pip install mlxtend

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [14]:
len(orders['articles'])

1362281

In [15]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()

In [16]:
orders_1hot = te.fit_transform(orders['articles'][0:136228])

orders_1hot = pd.DataFrame(orders_1hot, columns = te.columns_)

from mlxtend.frequent_patterns import fpgrowth
orders_fp = fpgrowth(orders_1hot, min_support=0.0001, max_len=2,
                use_colnames=True)

from mlxtend.frequent_patterns import association_rules
ar = association_rules(orders_fp, 
                       metric="lift", 
                       min_threshold=1.0)

ar

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(351484002),(372860002),0.018858,0.028115,0.000903,0.047879,1.702976,0.000373,1.020758
1,(372860002),(351484002),0.028115,0.018858,0.000903,0.032115,1.702976,0.000373,1.013697
2,(484398001),(351484002),0.022323,0.018858,0.000918,0.041105,2.179696,0.000497,1.023200
3,(351484002),(484398001),0.018858,0.022323,0.000918,0.048657,2.179696,0.000497,1.027681
4,(751471001),(351484002),0.020767,0.018858,0.000800,0.038530,2.043129,0.000409,1.020460
...,...,...,...,...,...,...,...,...,...
1347233,(740519006),(902507001),0.000250,0.000191,0.000110,0.441176,2311.561086,0.000110,1.789132
1347234,(556560001),(556560011),0.002444,0.000330,0.000147,0.060060,181.819152,0.000146,1.063546
1347235,(556560011),(556560001),0.000330,0.002444,0.000147,0.444444,181.819152,0.000146,1.795600
1347236,(818403003),(794320003),0.000272,0.000418,0.000213,0.783784,1873.215742,0.000213,4.623065


In [17]:
for i in range(1, 10):
    print(i)
    te = TransactionEncoder()

    orders_1hot = te.fit_transform(orders['articles'][136228*i:136228*(i+1)])

    orders_1hot = pd.DataFrame(orders_1hot, columns = te.columns_)

    from mlxtend.frequent_patterns import fpgrowth
    orders_fp = fpgrowth(orders_1hot, min_support=0.0001, max_len=2,
                    use_colnames=True)

    from mlxtend.frequent_patterns import association_rules
    new_ar= association_rules(orders_fp, 
                           metric="lift", 
                           min_threshold=1.0)
    ar = pd.concat([ar, new_ar])

1
2
3
4
5
6
7
8
9


In [27]:
clean_ar = ar.copy().sort_values(by=['support', 'lift'], ascending=False).drop_duplicates(subset=['antecedents', 'consequents'])
clean_ar

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
15557,(706016002),(706016001),0.034061,0.044506,0.012178,0.357543,8.033545,1.066221e-02,1.487250
15556,(706016001),(706016002),0.044506,0.034061,0.012178,0.273627,8.033545,1.066221e-02,1.329812
59318,(610776001),(610776002),0.027726,0.036534,0.010262,0.370135,10.131154,9.249272e-03,1.529638
59319,(610776002),(610776001),0.036534,0.027726,0.010262,0.280892,10.131154,9.249272e-03,1.352056
376426,(689109001),(692930001),0.013110,0.012163,0.008845,0.674692,55.468889,8.685998e-03,3.036620
...,...,...,...,...,...,...,...,...,...
20697,(787946002),(572998001),0.009719,0.010556,0.000103,0.010574,1.001723,1.767425e-07,1.000018
214219,(565379001),(687034009),0.015900,0.006460,0.000103,0.006464,1.000583,5.992003e-08,1.000004
214218,(687034009),(565379001),0.006460,0.015900,0.000103,0.015909,1.000583,5.992003e-08,1.000009
1213168,(717779001),(706016002),0.003017,0.034061,0.000103,0.034063,1.000080,8.190508e-09,1.000003


In [40]:
clean_ar.to_csv('~/CIS_4496_Project/data/clean_ar.csv')

In [29]:
def predict(antecedent, rules, max_results= 12):
    
    # get the rules for this antecedent
    preds = rules[rules['antecedents'] == antecedent]
    
    # a way to convert a frozen set with one element to string
    preds = preds['consequents'].apply(iter).apply(next)
    
    return preds[:max_results]

In [43]:
preds = predict({df_t['article_id'][5]}, clean_ar)
preds

/tmp/ipykernel_21834/1315413251.py:9: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return preds[:max_results]


2054     685687004
2056     685687003
16340    685687002
2068     706016001
2292     673677002
2202     610776002
2242     573716012
2070     759871002
2244     562245001
2204     610776001
2130     448509014
2072     706016002
Name: consequents, dtype: int64

In [41]:
df_c = pd.read_csv('~/CIS_4496_Project/data/raw/customers.csv')
df_c

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...
...,...,...,...,...,...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,NaN,NaN,ACTIVE,NONE,24.0,7aa399f7e669990daba2d92c577b52237380662f36480b...
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,NaN,NaN,ACTIVE,NONE,21.0,3f47f1279beb72215f4de557d950e0bfa73789d24acb5e...
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,1.0,1.0,ACTIVE,Regularly,21.0,4563fc79215672cd6a863f2b4bf56b8f898f2d96ed590e...
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,1.0,1.0,ACTIVE,Regularly,18.0,8892c18e9bc3dca6aa4000cb8094fc4b51ee8db2ed14d7...


In [98]:
predict({df_t['article_id'][4]}, clean_ar).empty

/tmp/ipykernel_21834/1315413251.py:9: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return preds[:max_results]


True

In [48]:
no_ct = list(set(df_c['customer_id']).symmetric_difference(set(np.unique(df_t['customer_id']))))

In [61]:
most_popular = list(df_t['article_id'].value_counts().index[0:12])

In [62]:
submission = pd.DataFrame(columns=['customer_id', 'prediction'])

In [104]:
customer_id = []
prediction = []

from datetime import datetime
print(datetime.now())

i = 0
for c, alist in zip(orders['customer_id'], orders['articles']):
    if (i % 1000 == 0):
        print(datetime.now(), i)
    i += 1
    
    customer_id.append(c)
    
    predictor = False
    for a in alist:
        if predict({a}, clean_ar).empty == False:
            prediction.append(predict({a}, clean_ar))
            predictor = True
            break
    if predictor == False:
        prediction.append(most_popular)
    
    

for c in no_ct:
    if (i % 1000 == 0):
        print(datetime.now(), i)
    i += 1
    
    customer_id.append(c)
    prediction.append(most_popular)

2023-03-28 22:55:12.105458
2023-03-28 22:55:12.105889 0


/tmp/ipykernel_21834/1315413251.py:9: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return preds[:max_results]


2023-03-28 23:09:44.170292 1000
2023-03-28 23:24:35.461231 2000
2023-03-28 23:39:17.630040 3000
2023-03-28 23:53:26.856652 4000
2023-03-29 00:07:00.956854 5000
2023-03-29 00:21:08.146022 6000
2023-03-29 00:36:03.424303 7000
2023-03-29 00:50:59.492186 8000
2023-03-29 01:05:34.399445 9000
2023-03-29 01:19:38.537816 10000
2023-03-29 01:34:18.396756 11000
2023-03-29 01:48:35.097464 12000
2023-03-29 02:03:11.080195 13000
2023-03-29 02:18:05.361797 14000
2023-03-29 02:32:37.924609 15000
2023-03-29 02:46:59.869989 16000
2023-03-29 03:01:37.650686 17000
2023-03-29 03:16:13.703707 18000
2023-03-29 03:31:01.560166 19000
2023-03-29 03:45:47.486028 20000
2023-03-29 04:00:31.664604 21000
2023-03-29 04:15:10.382421 22000
2023-03-29 04:30:12.805249 23000
2023-03-29 04:45:12.729551 24000
2023-03-29 05:00:02.657235 25000
2023-03-29 05:15:09.427083 26000
2023-03-29 05:31:00.190796 27000
2023-03-29 05:45:52.945761 28000
2023-03-29 06:00:40.239723 29000
2023-03-29 06:15:00.743293 30000
2023-03-29 06:29:52

KeyboardInterrupt: 

In [105]:
submission['customer_id'] = customer_id[0:25000]
submission['prediction'] = prediction[0:25000]
submission

,customer_id,prediction
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,373021 487722001 500 667641001 508 ...
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,2057 685687001 1250 685687004 16343 ...
2,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,8475 688873009 8493 688873008 8502 ...
3,0008968c0d451dbc5a9968da03196fe20051965edde741...,14963 762205001 1269380 529841002 1496...
4,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,19676 501820004 19673 501820001 19627 ...
...,...,...
24995,d545900e90aef586a118d1d7e3097d42c2d31735304ea6...,721105 661162002 754641 661162001 7546...
24996,d54626a78c3b750ece7ab4d769e8b341061f9cad871a2a...,1244127 540701008 6895 413707001 7401...
24997,d5480eb6e69180fe904ac415374f311a1c9297c81d4546...,1309878 685816002 1309876 636587002 1309...
24998,d54b955cbbfce16c1a685add2561ff73fadde0741eab11...,632370 617919021 1184008 512251004 1184...
